# a6 - Tweeter Sentiment

In this assignment you will write a program to perform simple [sentiment analysis](https://en.wikipedia.org/wiki/Sentiment_analysis) of Twitter data&mdash;that is, determining the "attitude" or "emotion" (e.g., how "positive", "negative", "joyful", etc) of tweets made by a particular Twitter user. Sentiment analysis is a fascinating field: researchers have shown that the "mood" of Twitter communication [reflects biological rhythms](http://www.nytimes.com/2011/09/30/science/30twitter.html) and can even be used to [predict the stock market](http://arxiv.org/pdf/1010.3003&embedded=true). The particular analysis you'll be performing is inspired by an investigation of [personal vs. organizational tweets](http://varianceexplained.org/r/trump-tweets/) (which has become less amusing over time).

You will be implementing a Python program that performs this analysis on **real data** taken directly from a Twitter user's timeline. In the end, your script will produce output similar to the following:

```
EMOTION       % WORDS  EXAMPLE WORDS                     HASHTAGS
positive      6.16%    learn, faculty, happy             #accesstoinfoday, #indigenouspeoplesday, #idealistfair
trust         3.08%    school, faculty, happy            #indigenouspeoplesday, #diversity
anticipation  2.53%    happy, top, ready                 #indigenouspeoplesday, #informatics, #info340
joy           1.76%    happy, peace, deal                #indigenouspeoplesday, #accesstoinfoday
surprise      0.99%    deal, award, surprised            #suzzallolibrary, #nobrainer
negative      0.88%    fall, rejection, outstanding        
sadness       0.55%    fall, rejection, problem            
disgust       0.44%    rejection, weird, finally           
fear          0.44%    rejection, surprise, problem        
anger         0.33%    rejection, disaster, involvement  #mlis
```

(The "hashtags" column is optional extra credit).

Fill in the below code cells as specified. Note that cells may utilize variables and functions defined in previous cells; we should be able to use the `Kernal > Restart & Clear All` menu item followed by `Cell > Run All` to execute your entire notebook and see the correct output.

## The Data
You'll be working with two different pieces of data for this assignment.

First, you'll be loading tweet data taken directly from [Twitter's API](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline). You can find an example of this tweet data in the **`uw_ischool_sample.py`** file inside the `data/` folder. The below cell will import this data as a variable `SAMPLE_TWEETS` from the provided _module_ file:

In [1]:
# import from uw_ischool_sample file in the `data/` package (folder)
from data.uw_ischool_sample import SAMPLE_TWEETS

The data is represented as one giant **list of dictionaries**: the **list** contains a sequence of **dictionaries**, where each dictionary represents a tweet. Each dictionary contains many different _values_, some of which themselves may be dictionaries!

Print out the first three elements from the `SAMPLE_TWEETS` list to see what information can be found in each. The most relevant value is the `"text"` of the tweet.
- The Twitter API actually provides a lot more information about each tweet; I've stripped it down to only the most important values for readability. Each dictionary in the example is a proper subset of the full data you'd get from Twitter.
- Because of the source of the sentiment data, your analysis will be biased and only support English-language speakers. Nevertheless, Twitter is an international community so you may encounter non-English characters and words. You'll be working with real-world data and it will be messy!

In [2]:
print(SAMPLE_TWEETS[:3])

[{'created_at': 'Mon Oct 10 18:39:51 +0000 2016', 'retweet_count': 9, 'entities': {'hashtags': [{'indices': [20, 41], 'text': 'IndigenousPeoplesDay'}]}, 'user': {'screen_name': 'UW_iSchool'}, 'text': 'RT @UWAPress: Happy #IndigenousPeoplesDay https://t.co/YmU9e9lj7v'}, {'created_at': 'Mon Oct 10 18:00:00 +0000 2016', 'retweet_count': 0, 'entities': {'hashtags': [{'indices': [16, 29], 'text': 'IdealistFair'}]}, 'user': {'screen_name': 'UW_iSchool'}, 'text': "We'll be at the #IdealistFair this evening on the Seattle U. campus. Come and learn about our graduate programs: https://t.co/et1HrQshmr"}, {'created_at': 'Mon Oct 10 15:10:36 +0000 2016', 'retweet_count': 1, 'entities': {'hashtags': []}, 'user': {'screen_name': 'UW_iSchool'}, 'text': 'RT @iYouthUW: iYouth Tips for 1st\xa0Years https://t.co/K4SCIEhJ8k https://t.co/p4lbC6Jb5o'}]


The second piece of data you'll be working with is a set of **word-sentiments**&mdash;a set of English-language words and what emotions (e.g., "joy", "anger") [are associated with them](http://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm).

- The [nltk](https://github.com/nltk/nltk/wiki/Sentiment-Analysis) library you used in a previous assignment does support sentiment analysis. However, for practice and extendability you'll be doing a more "manual" analysis using the provided data file for this assignment. You must _not_ use that library in this assignment!

`import` the word sentiments as a variable **`SENTIMENTS`** from the **`data.sentiments_nrc`** module. You should also import the **`EMOTIONS`** variable provided by the same module: this is a _list_ of possible emotions (as strings). You can inspect the variables (e.g., print them out) to confirm that you have imported them, as well as to see what they look like.

- Notice that `SENTIMENTS` is a _dictionary_ whose keys are English-language words (e.g., "sunshine", "rainy"). Each key points to a _another dictionary_, whose keys are _emotions_ (e.g., "happy", "sad") that go with that word. Thus `SENTIMENTS['sunshine']` will give you a dictionary of emotions for the word "sunshine", and `SENTIMENTS['sunshine'].get("happy")` will let you know if the word "sunshine" is a happy word or not.

In [3]:
from data.sentiments_nrc import SENTIMENTS
from data.sentiments_nrc import EMOTIONS

print(SENTIMENTS["sunshine"])

print(SENTIMENTS['sunshine'].get("happy"))
print(EMOTIONS)

{'positive': 1, 'joy': 1}
None
['positive', 'negative', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']


## Text Sentiment
All of the sentiment analysis will be based on the individual _words_ in the text. Thus you will need to will determine which words in a tweet have which sentiments.

- Note that the assignment explicitly does _not_ tell you what to name functions, what arguments they should take or values they should return: your task is to determine appropriate functions and arguments from the (guided) requirements! Use multiple functions for clarity and give them all informative names! Try to make each function "stand alone"--so that it takes any information it needs as argumets, and then returns any information that it determines to be used elsewhere.

Define a function that take a tweet's text (a string) and splits it up into a list of individual words.

- You must **not** use the `nltk` library to tokenize words. Instead, your analysis must split up the text using the [regular expression](https://www.regular-expressions.info/) **`"\W+"`** as a separator to "split up" the words by (rather than just a blank space). You can do this by using the [re.split()](https://docs.python.org/3/library/re.html#re.split) function (from the `re` module)--pass the function the separator string and then the string you wish to split, and it will return the split up list. Using `"\W+"` as a separator will cause your spitting to exclude punctuation and provide a reasonable (but not perfect!) list of words to consider. 

- All of the words in the sentiment dictionary are _lower case_, so you'll need to **map** your resulting words to be lower case. You will also need to **filter** out any words that have 1 letter or fewer. Your function must use a **list comprehension** to do both of these.

To test your work, the string `"Amazingly, I prefer a #rainy day to #sunshine."` should produce a list with 6 lower-case words in it.

In [4]:
# take in a string and return a list with the words with more than 1 letter in the lower-case.
import re
def longword(word):
    return len(word) > 1
def get_words(string):
    wordlist = re.split(r'\W+', string.lower())
    result = list(filter(longword, wordlist))
    return result

get_words("Amazingly, I prefer a #rainy day to #sunshine.")

['amazingly', 'prefer', 'rainy', 'day', 'to', 'sunshine']

Define a function that **filters** a list of words to get a list of only those words that contain a specific emotion. Your function must use a **list comprehension** to do this.
- You can determine whether a word has a particular emotion by looking it up in the imported `SENTIMENTS` variable. Use the word as the "key" to find the dictionary of emotions for that word, and then use the emotion as the key to _that_ dictionary to determine if the word has it. 
    
    Do **not** use the `in` operator or a loop to search the list of keys one by one. Instead, use bracket notation or the `get()` method to "look up" a key directly (this is more efficient and is why the emotions are nested dictionaries in the first place!).
    
For testing, the `"positive"` words extracted from `"Amazingly, I prefer a #rainy day to #sunshine."` are `["amazingly", "prefer", "sunshine"]`.

In [5]:
# take in the string and the list of specific emotions. Return a list of words that has the emotion.
def get_emotion(words_list, emotion):
    #loop through the list to see if each of the words has a emotion.
    emotion_words = []
    for word in words_list:
        #If the word is in the SENTIMNTS dictionary
        if SENTIMENTS.get(word):
             if SENTIMENTS[word].get(emotion) == 1:
                 #append the emotion in a new list
                 emotion_words.append(word)
    return emotion_words


test_string = "Amazingly, I prefer a #rainy day to #sunshine."
test_list = get_words(test_string)
get_emotion(test_list, 'positive')


['amazingly', 'prefer', 'sunshine']

Define a function that determines which words from a list have _each_ emotion (i.e., the "emotional" words), producing a dictionary of that information. For example, the words extracted from `"Amazingly, I prefer a #rainy day to #sunshine."` should produce a dictionary that looks like:

```
{
 'positive': ['amazingly', 'prefer', 'sunshine'],
 'negative': [],
 'anger': [],
 'anticipation': [],
 'disgust': [],
 'fear': [],
 'joy': ['amazingly', 'sunshine'],
 'sadness': ['rainy'],
 'surprise': ['amazingly'],
 'trust': ['prefer']
}
```
    
(Note the empty lists for emotions that have no matching words).
    
- You can use the imported `EMOTIONS` variable to have a list of emotions to iterate through.
- Use the function you defined in the previous step to help you out!
- Using a [dictionary comprehension](https://www.smallsurething.com/list-dict-and-set-comprehensions-by-example/) is a nice way to do this, but is not required&mdash;using a regular loop is fine.

In [6]:
def emotion_dictionary(string):
    emotion_dictionary = {}
    words = get_words(string)
    #loop through the emotions
    for emotion in EMOTIONS:
        #find the words for each emotion
        emotion_word = get_emotion(words, emotion)
        emotion_dictionary[emotion] = emotion_word
    return emotion_dictionary

emotion_dictionary(test_string)

    #['positive', 'negative', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']

{'positive': ['amazingly', 'prefer', 'sunshine'],
 'negative': [],
 'anger': [],
 'anticipation': [],
 'disgust': [],
 'fear': [],
 'joy': ['amazingly', 'sunshine'],
 'sadness': ['rainy'],
 'surprise': ['amazingly'],
 'trust': ['prefer']}

Define a function that gets a list of the "most common" words in a list. This should be a new list containing each word in the original list, in descending order by how many times that word appears in the orignal list.

- You can determine the _frequency_ (number of occurrences) of a word with a similar process to what you did with digits in the previous assignment.
- You can use the `sorted()` function to [sort](https://wiki.python.org/moin/HowTo/Sorting) the items in a frequency dictionary. This function accepts a [**key**](https://wiki.python.org/moin/HowTo/Sorting#Key_Functions) argument which is passed the name of a "mapping" function that maps an element (e.g., an item tuple) into the value you wish to sort by (e.g., the element of the tuple). An anonymous lambda function works well for this callback, but is not necessary.

You can test this function with any list of "words" with repeated entries: `['a','b','c','c','c','a']` for example.

In [7]:
def count_frequency(wordlist):
    wordlist = sorted(wordlist)
    start_word = wordlist[0]
    wordcount = 1
    new_list = []
    for element in wordlist[1:]:
        if element == start_word:
            wordcount += 1
        else:
            new_list.append((start_word, wordcount))
            start_word = element
            wordcount = 1
    new_list.append((start_word, wordcount))
    return new_list

def get_word_from_tuple(list_tuple):
    list_word = []
    for tup in list_tuple:
        list_word.append(tup[0])
    return list_word

def sort_frequency(wordlist):
    frequency_list = count_frequency(wordlist)
    word_frequency = sorted(frequency_list, key=lambda frequency: frequency[1], reverse = True)
    result = get_word_from_tuple(word_frequency)
    return result


testlist = ['a','b','c','c','c','a']

print(sort_frequency(testlist))


['c', 'a', 'b']


## Tweet Statistics
Once you are able to determine the sentiment of an individual string of text (e.g., a single tweet's content), you can analyze an entire set of tweets from a user's timeline. Note that I am _not_ telling you how to break this process up into individual functions&mdash;you should be able to determine that on your own by this point!

Define a function (e.g., `analyze_tweets()`) that takes as an argument a **list** of tweet data (with the same structure as the imported `SAMPLE_TWEETS` variable), and _returns_ the data of interest to display in a table like the one at the very top of the notebook. In particular, you'll need to produce the following information **for each emotion**:

1. The percentage of words _across all tweets_ that have that emotion
2. The most common words _across all tweets_ that have that emotion (in order!)

(Think carefully: should this data be stored in a _list_ or a _dictionary_?)

Some tips for this task:

- You can and should create some "helper" functions to break up this task even further. You can define those functions in the same notebook cell or you may insert additional cells below these instructions.

- You'll need to use your previous functions to get the _list of words_ and _dictionary of emotional words_ for each tweet. I recommend you assign the results of these methods as **new values** of the respective tweet dictionary (so each tweet dictionary would gain a `words` key and a `sentiments` key, for example).

- In order to get the percentage of emotional words, divide the number of words that have that emotion by the total number of words _across all the tweets_. Counting how many total words are in the tweet set is a **reducing** operation. As an **optional advanced requirement**, do this using the `reduce()` function.

- For each emotion, you'll need to get a list of the words _across all the tweets_ that have that emotion (in order to determine how many there are for the percentage, as well as which are most common). This is another **reducing** operation. It is possible to use the built-in `sum()` function, or as an **optional advanced requirement** use the `reduce()` function.

You can test your function by passing in the `SAMPLE_TWEETS` variable as an argument and checking if your returned data has the same numbers as in the table at the top of the page. Note that only the first 3 most common words are listed in the example (and may be in a different order in the case of frequency ties!).
- For testing, my solution counts 909 total words in the sample; 56 of those words are "positive".

In [39]:
# Get the text of tweet from the SAMPLE_TWEETS
def get_tweets(sample_tweets):
    tweets = []
    for dictionary in sample_tweets:
        tweet = dictionary['text']
        tweets.append(tweet)
    return tweets

# for all the tweets, add all the words into one list
def get_word_from_tweet(list_tweets):
    word_list = []
    for tweet in list_tweets:
        word_list += get_words(tweet)
    return word_list

def frequency_dictionary(tweet_word ,emotion,total_length):
        word_have_emotion = get_emotion(tweet_word, emotion)
        emotion_length = len(word_have_emotion)
        percentage = round(emotion_length/total_length, 4)
        word_frequency = sort_frequency(word_have_emotion)
        dictionary= {'emotion':emotion, 'percentage': "{:.2%}".format(percentage), "common words": word_frequency[:3]}
        return dictionary


#Main
def analyze_tweets(sample_tweets):
    tweets_list = get_tweets(sample_tweets)
    tweets_word = get_word_from_tweet(tweets_list)
    total_length = len(tweets_word)
    #Find the words that have a specific emotion:
    analyze_result = []
    for emotion in EMOTIONS:
        dictionary = frequency_dictionary(tweets_word, emotion,total_length)
        analyze_result.append(dictionary)
    analyze_tweet = sorted(analyze_result, key=lambda  emotion: emotion['percentage'], reverse = True)
    return analyze_tweet

analyze_tweet = analyze_tweets(SAMPLE_TWEETS)
print(analyze_tweet)



    


[{'emotion': 'positive', 'percentage': '6.16%', 'common words': ['faculty', 'learn', 'happy']}, {'emotion': 'trust', 'percentage': '3.08%', 'common words': ['school', 'faculty', 'happy']}, {'emotion': 'anticipation', 'percentage': '2.53%', 'common words': ['happy', 'ready', 'top']}, {'emotion': 'joy', 'percentage': '1.76%', 'common words': ['happy', 'award', 'deal']}, {'emotion': 'surprise', 'percentage': '0.99%', 'common words': ['award', 'deal', 'disaster']}, {'emotion': 'negative', 'percentage': '0.88%', 'common words': ['fall', 'boring', 'disaster']}, {'emotion': 'sadness', 'percentage': '0.55%', 'common words': ['fall', 'disaster', 'problem']}, {'emotion': 'disgust', 'percentage': '0.44%', 'common words': ['disaster', 'finally', 'rejection']}, {'emotion': 'fear', 'percentage': '0.44%', 'common words': ['disaster', 'problem', 'rejection']}, {'emotion': 'anger', 'percentage': '0.33%', 'common words': ['disaster', 'involvement', 'rejection']}]


***Optional extra credit***: For each emotion, also calculate the most common [**hashtags**](https://en.wikipedia.org/wiki/Hashtag) _across all tweets_ associated with that emotion. This means that out of all the weets that have _at least one word with that emotion_, what are the most common hashtags they use?

- The Twitter data for each tweet includes a _list_ containing the hashtags found in that tweet&mdash;you should **NOT** try and search the tweet text for `#` symbols. These hashtags can be found in the `['entities']['hashtags'][i]['text']` element of each tweet&mdash;that is, the `'text'` key from _each_ element in the _list_ of the `'hashtags'` key in the `'entities'` dictionary of the tweet. See the `uw_school.json` example file to see this structure more clearly.

    (You might use a _list comprehension_ to "flatten" this complex nesting structure into just a list of hashtag words).

- Since hashtags are just words, you can use your function for finding the most common words to find the most common hashtags!

- You can build your analysis into your previous `analyze_tweets()` function, or you can determine the hashtags using a second function (e.g., in the below cell).


## Displaying Tweets 

Once you've analyzed the tweets, you will need to _display_ that information as a printed table (as in the example at the top of the page).

Define another function to display this table (your function should take as an argument the data structure returned from your "analysis" function).

This function will need to print out the table. Using the [string formatting](https://docs.python.org/3/library/string.html#format-examples) language (via the **`.format()`** string method) makes it possible to have equally sized "columns" of data. For more examples, [this tutorial](https://www.digitalocean.com/community/tutorials/how-to-use-string-formatters-in-python-3) is pretty good (check out the "Padding Variable Substitutions" section).


A few notes about formatting this output:

- For your reference, the example table at the top of the page uses `14` characters for the first column, `11` characters for the second,  `35` for the third, and the "remainder" for the fourth. You are not required to match these numbers.

- The percentage should be formatted with two decimals of precision (e.g., `1.23%`).

- Both the example sentiment words (and the hashtags, if you include them) should be outputted as a _comma-separated list_ with spaces between them&mdash;and no square brackets). The `join()` string method is good for converting lists to formatted strings. Both lists should also be limited to the 3 most common items for space.

- Make sure to include `#` in front of the hashtags!

In [38]:
def table_display(analyze_tweet):
    print("{0:<14} {1:<11} {2:<35}".format('EMOTION','% WORDS','EXAMPLE WORDS'))
    for dictionary in analyze_tweet:
        commonword_list = dictionary['common words']
        str = ","
        seq = (commonword_list[0],commonword_list[1],commonword_list[2])
        commonword = str.join(seq)
        print("{0:<14} {1:<11} {2:<35}".format(dictionary["emotion"], dictionary['percentage'],commonword))

print(table_display(analyze_tweet))

EMOTION        % WORDS     EXAMPLE WORDS                      
positive       6.16%       faculty,learn,happy                
trust          3.08%       school,faculty,happy               
anticipation   2.53%       happy,ready,top                    
joy            1.76%       happy,award,deal                   
surprise       0.99%       award,deal,disaster                
negative       0.88%       fall,boring,disaster               
sadness        0.55%       fall,disaster,problem              
disgust        0.44%       disaster,finally,rejection         
fear           0.44%       disaster,problem,rejection         
anger          0.33%       disaster,involvement,rejection     
None


## Getting Live Data
This is all good and well, but the real payoff is to be able to see the sentiments of tweets taken directly from the Twitter feed of real users!

Define _another_ function (e.g., `get_user_tweets()`) that takes as an argument a Twitter username (as a string) then returns a list of dictionaries representing the tweets made by that user.

I have set up a [proxy](https://en.wikipedia.org/wiki/Proxy) that you can use to send requests to [Twitter's API](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline) and access the data. You can thus get tweet data for a specific username at the URL:

```
https://faculty.washington.edu/joelross/proxy/twitter/timeline/?screen_name=SCREEN_NAME
```

(replacing `SCREEN_NAME` with the Twitter username).

- _WARNING:_ The proxy I have set up is _rate-limited_ so that it can only accept 900 requests every 15 minutes. If all 40 students are working rapidly on the assignment at the same time, you may find yourself needing to wait a few minutes and try again. You are alternatively welcome to set up your own developer account and API keys; just make sure you don't put the keys under version control and upload them to GitHub!

You can download the timeline data from Twitter using the [requests](http://docs.python-requests.org/en/master/user/quickstart/) module: use **`requests.get()`** to get a response from the above URL. Then call the **`.json()`** method on that response to extract the _list of dictionaries_ that your function will return. See [Chapter 12](https://infx511.github.io/accessing-web-apis.html#accessing-web-apis-1) of the course text for more details and examples.

In [59]:
import requests
def get_user_tweets(username):
    url = 'https://faculty.washington.edu/joelross/proxy/twitter/timeline/?screen_name=' + username
    response = requests.get (url)
    tweets = response.json()
    return tweets

Finally, define one last "main" function that will [prompt the user](https://docs.python.org/3/library/functions.html#input) for a Twitter username. The function should then call your `get_user_tweets()` function to fetch the tweets, and pass the returned tweet data into your "analyze" and "display" functions in order to display your sentiment analysis of the user's timeline. 

**ADDITIONALLY**, your main function should check that `if` the user specifies `SAMPLE` (all caps) as the username, the function should instead show the analysis for the `SAMPLE_TWEETS` (this will help us out with grading).

In [63]:
def main_function():
    username = input('Enter the user name: ')
    if username == "SAMPLE":
        data = SAMPLE_TWEETS
    else:
        data = get_user_tweets(username)
    analyze_data = analyze_tweets(data)
    return analyze_data

main_function()

Enter the user name: UW_iSchool


[{'emotion': 'positive',
  'percentage': '7.33%',
  'common words': ['director', 'launch', 'learn']},
 {'emotion': 'anticipation',
  'percentage': '2.36%',
  'common words': ['launch', 'public', 'plan']},
 {'emotion': 'trust',
  'percentage': '1.89%',
  'common words': ['director', 'real', 'center']},
 {'emotion': 'negative',
  'percentage': '0.95%',
  'common words': ['fight', 'liaison', 'quote']},
 {'emotion': 'joy',
  'percentage': '0.71%',
  'common words': ['pleased', 'proud', 'special']},
 {'emotion': 'surprise',
  'percentage': '0.71%',
  'common words': ['catch', 'quote', 'surprising']},
 {'emotion': 'fear',
  'percentage': '0.47%',
  'common words': ['fight', 'watch']},
 {'emotion': 'anger', 'percentage': '0.24%', 'common words': ['fight']},
 {'emotion': 'disgust', 'percentage': '0.24%', 'common words': ['interested']},
 {'emotion': 'sadness', 'percentage': '0.24%', 'common words': ['interested']}]

In the below cell, call your main function to analyze the timeline of a Twitter user. You can even call it multiple times to compare the sentiments of different users. Are the current sentiments of the [iSchool](https://twitter.com/uw_ischool) and [CSE](https://twitter.com/uwcse) different in interesting ways?

In [62]:
main_function()

Enter the user name: uwcse


[{'emotion': 'positive',
  'percentage': '5.31%',
  'common words': ['join', 'expert', 'full']},
 {'emotion': 'trust',
  'percentage': '3.14%',
  'common words': ['expert', 'professor', 'school']},
 {'emotion': 'joy',
  'percentage': '1.69%',
  'common words': ['create', 'excitement', 'food']},
 {'emotion': 'negative',
  'percentage': '1.45%',
  'common words': ['coma', 'falling', 'forget']},
 {'emotion': 'anticipation',
  'percentage': '0.97%',
  'common words': ['excitement', 'improve', 'time']},
 {'emotion': 'anger',
  'percentage': '0.72%',
  'common words': ['kick', 'kicking', 'mosquito']},
 {'emotion': 'surprise',
  'percentage': '0.72%',
  'common words': ['catch', 'excitement']},
 {'emotion': 'fear',
  'percentage': '0.48%',
  'common words': ['coma', 'volunteer']},
 {'emotion': 'sadness',
  'percentage': '0.48%',
  'common words': ['coma', 'falling']},
 {'emotion': 'disgust', 'percentage': '0.24%', 'common words': ['mosquito']}]